In [1]:
# from transformers import BertTokenizer, BertModel
import torch
import pandas as pd
import numpy as np
import math

In [2]:
import data_file
# male_list = data_file.male_list
# female_list = data_file.female_list
female_professions = data_file.female_professions
male_professions = data_file.male_professions

In [3]:
# models

# model1
from transformers import BertTokenizer, BertModel
import torch
tokenizer1 = BertTokenizer.from_pretrained('bert-base-uncased')
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model1 = BertModel.from_pretrained("bert-base-uncased")
# model1.cuda()

# model2
from transformers import BertTokenizer, BertModel
import torch
tokenizer2 = BertTokenizer.from_pretrained('bert-large-uncased')
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model2 = BertModel.from_pretrained("bert-large-uncased")
# model2.cuda()

# model3
from transformers import RobertaTokenizer, RobertaModel
import torch
tokenizer3 = RobertaTokenizer.from_pretrained('roberta-base')
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model3 = RobertaModel.from_pretrained("roberta-base")
# model3.cuda()

# model4
from transformers import DistilBertTokenizer, DistilBertModel
import torch
tokenizer4 = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model4 = DistilBertModel.from_pretrained("distilbert-base-uncased")
# model4.cuda()

# model5
from transformers import BertTokenizer, BertModel
import torch
tokenizer5 = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model5 = BertModel.from_pretrained("bert-base-multilingual-uncased")
# model5.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transf

In [4]:
models = [model1, model2, model3, model4, model5]
tokenizers = [tokenizer1, tokenizer2, tokenizer3, tokenizer4, tokenizer5]

In [5]:
female_list = list(female_professions.label1)
male_list = list(male_professions.label1)

In [6]:
# # gendered_word_list
# gendered_word_list = [["she", "he"],["female", "male"], ["woman", "man"], ["girl", "boy"], ["sister", "brother"], ["daughter", "son"], ["mother", "father"]]
XX = ["female", "woman", "girl", "sister", "daughter", "mother", "aunt", "grandmother", "daughter"]
YY = ["male", "man", "boy", "brother", "son", "father", "uncle", "grandfather", "son"]

In [7]:
templates = ["this is the _.", "that is the _.", "there is the _.", "here is the _.", "the _ is here.", "the _ is there.", "the _ is a person.", "it is the _.","the _ is a [MASK].", "the _ is an engineer.", "the _ is a nurse with superior technical skills."]

In [8]:
temps = ["temp1", "temp2", "temp3", "temp4","temp5", "temp6", "temp7", "temp8", "temp9", "temp10", "temp11"]

In [9]:
len(templates)

11

In [81]:
def get_index(sentence, word, tokenizer):
    if tokenizer == tokenizers[2]:
        word = " "+word
    toks = tokenizer(sentence).input_ids
    wordpieces = tokenizer(word).input_ids
    word = wordpieces[1] # use first wordpiece
    for i, t in enumerate(toks):
        if t == word:
            return i

In [69]:
# SWP first embedding
def sentence_embedding(template, word, model_ind):
    sentence = template.replace("_", word)
    inputs = tokenizers[model_ind](sentence, return_tensors="pt")
    outputs = models[model_ind](**inputs)
    last_hidden_states = outputs.last_hidden_state
    token_embeddings = last_hidden_states
    input_mask_expanded = inputs.attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    start = get_index(sentence, word, tokenizers[model_ind])
#     print(start)
    embeddings = token_embeddings[0][start]
#     print(embeddings.shape)
    return embeddings.cpu().detach().numpy()


# # CLS embedding
# def sentence_embedding(template, word, model_ind):
#     sentence = template.replace("_", word)
#     inputs = tokenizers[model_ind](sentence, return_tensors="pt")
#     outputs = models[model_ind](**inputs)
#     last_hidden_states = outputs.last_hidden_state
#     token_embeddings = last_hidden_states
#     return token_embeddings[0][0].cpu().detach().numpy()

In [45]:
def cossim(x, y):
    return np.dot(x, y) / math.sqrt(np.dot(x, x) * np.dot(y, y))

In [30]:
def construct_cossim_lookup(XY, AB):
    """
    XY: mapping from target string to target vector (either in X or Y)
    AB: mapping from attribute string to attribute vectore (either in A or B)
    Returns an array of size (len(XY), len(AB)) containing cosine similarities
    between items in XY and items in AB.
    """

    cossims = np.zeros((len(XY), len(AB)))
    for xy in XY:
        for ab in AB:
#             print(XY[xy].shape)
#             print(AB[ab].shape)
            cossims[xy, ab] = cossim(XY[xy], AB[ab])
    return cossims


In [13]:
def convert_keys_to_ints(X, Y):
    return (
        dict((i, v) for (i, (k, v)) in enumerate(X.items())),
        dict((i + len(X), v) for (i, (k, v)) in enumerate(Y.items())),
    )

In [14]:

def s_XAB(A, s_wAB_memo):
    return s_wAB_memo[A].sum()

In [15]:
def s_wAB(X, Y, cossims):
    """
    Return vector of s(w, A, B) across w, where
        s(w, A, B) = mean_{a in A} cos(w, a) - mean_{b in B} cos(w, b).
    """
#     print((cossims[X, :].mean(axis=0) - cossims[Y, :].mean(axis=0)).shape)
    return cossims[X, :].mean(axis=0) - cossims[Y, :].mean(axis=0)

In [16]:
# def s_wAB_df(X, Y, cossims):
#     """
#     Return vector of s(w, A, B) across w, where
#         s(w, A, B) = mean_{a in A} cos(w, a) - mean_{b in B} cos(w, b).
#     """
# #     print(cossims[:, A].mean(axis=1))
#     df1 = pd.DataFrame(cossims[X, :].mean(axis=0))
#     df2 = pd.DataFrame(cossims[Y, :].mean(axis=0))
#     return df1, df2


def s_XAB_df(A, B, s_wAB_memo):
    df1 = pd.DataFrame(s_wAB_memo[A])
    df2 = pd.DataFrame(s_wAB_memo[B])
    return df1, df2

In [17]:
def s_XYAB(A, B, s_wAB_memo):
    r"""
    Given indices of target concept X and precomputed s_wAB values,
    the WEAT test statistic for p-value computation.
    """
    return s_XAB(A, s_wAB_memo) - s_XAB(B, s_wAB_memo)

In [18]:
# def WEAT_test(X, Y, A, B, n_samples, cossims, parametric=False):
def WEAT_test(X, Y, A, B, n_samples, cossims):
    ''' Compute the p-val for the permutation test, which is defined as
        the probability that a random even partition X_i, Y_i of X u Y
        satisfies P[s(X_i, Y_i, A, B) > s(X, Y, A, B)]
    '''
    X = np.array(list(X), dtype=np.int)
    Y = np.array(list(Y), dtype=np.int)
    A = np.array(list(A), dtype=np.int)
    B = np.array(list(B), dtype=np.int)

    assert len(X) == len(Y)
    size = len(X)
    s_wAB_memo = s_wAB(X, Y, cossims=cossims)
#     print(s_wAB_memo)
    XY = np.concatenate((X, Y))

#     if parametric:
#     log.info('Using parametric test')
    s = s_XYAB(A, B, s_wAB_memo)
    return s


def convert_keys_to_ints(X, Y):
    return (
        dict((i, v) for (i, (k, v)) in enumerate(X.items())),
        dict((i + len(X), v) for (i, (k, v)) in enumerate(Y.items())),
    )

In [19]:
def get_effect_size(df1, df2, k=0):
    diff = (df1[k].mean() - df2[k].mean())
    std_ = pd.concat([df1, df2], axis=0)[k].std() + 1e-8
    return diff / std_

In [82]:
template_score_dict = {}
for ind, template in enumerate(templates):
    score_dict = {}
    attribute_template = template
    target_template = template
    for model_ind in range(len(models)):
#     for model_ind in [2]:
        X = {"x" + str(j): sentence_embedding(attribute_template, j, model_ind) for j in XX}
        Y = {"y" + str(j): sentence_embedding(attribute_template, j, model_ind) for j in YY}
        (X, Y) = convert_keys_to_ints(X, Y)
        XY = X.copy()
        XY.update(Y)
        X = np.array(list(X), dtype=np.int)
        Y = np.array(list(Y), dtype=np.int)
        AA = female_list
    #     print(AA)
    #     print(XX)
        BB = male_list

        A = {"a" + str(j): sentence_embedding(target_template, j, model_ind) for j in AA}
        B = {"b" + str(j): sentence_embedding(target_template, j, model_ind) for j in BB}

        (A, B) = convert_keys_to_ints(A, B)


        AB = A.copy()
        AB.update(B)

        cossims = construct_cossim_lookup(XY, AB)
        A = np.array(list(A), dtype=np.int)
        B = np.array(list(B), dtype=np.int)


        s_wAB_memo = s_wAB(X, Y, cossims=cossims)
        df1,df2 = s_XAB_df(A, B, s_wAB_memo)
        effect_size = get_effect_size(df1, df2)
        score_dict[model_ind] = effect_size
        print(score_dict[model_ind])
    template_score_dict[temps[ind]] = score_dict
# log.info("pval: %g", pval)

# log.info("computing effect size...")
# esize = effect_size(X, Y, A, B, cossims=cossims)
# log.info("esize: %g", esize)

0.6973097940653281
0.7161906404882316
0.7262587506258051
0.5753516779477148
0.5883735676816193
0.6380417927901232
0.76153891651494
0.4823233219439905
0.6796147934048773
0.5249359284857726
0.526830739334173
0.6381677074285752
0.4382724747196723
0.6247362147488852
0.4291889880253535
0.6571533112974566
0.5382523873979591
0.36734113973550936
0.6926390803026279
0.506632763896542
0.7697797690278761
0.7184686780742054
0.5897673638382551
0.7446589271969147
0.3822477115258905
0.6996148984503365
0.8555235679456379
0.557613127438477
0.7457103726224552
0.4132492406485047
0.4263691690025617
0.3640531398300667
0.6450763525446751
0.635424855593478
0.41847001821573504
0.8835565676885099
0.7449150193768985
0.4705317846764712
0.6829442280114368
0.4705093104906551
0.7320277574390526
0.270907732971357
0.635379735121305
0.6118890639630125
0.6433825610507188
0.49245037034174094
0.1771056308189551
0.6446082457235965
0.69898358864895
0.3437851683409121
0.5482461883741784
1.0580662591076295
0.8115032709178353


In [1]:
import json
a_file = open("Seat_SWP_templates_Language_models.json", "w")
json.dump(template_score_dict, a_file)
a_file.close()

NameError: name 'template_score_dict' is not defined